<a href="https://colab.research.google.com/github/p3bble123/ewha16-child-abuse-detection/blob/main/input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 tensorflow-hub opencv-python matplotlib

     |████████████████████████████████| 394.3 MB 11 kB/s 
     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
     |████████████████████████████████| 462 kB 59.7 MB/s 
     |████████████████████████████████| 2.9 MB 38.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing insta

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [5]:
# 어른 아이 구분 함수

def separate_adult(frame, current_keypoints, confidence_threshold):

  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  
  i = 0
  adult_index = 0
  max_height = 0

  for person in shaped:

    # confidence가 역치 이상일 때 키 추출
    if (person[5][2] > confidence_threshold) and (person[6][2] > confidence_threshold) and (person[11][2] > confidence_threshold) and (person[12][2] > confidence_threshold):
       
       m = (person[5][0]+person[6][0])/2, (person[5][1]+person[6][1])/2      # shoulder 중점
       n = (person[11][0]+person[12][0])/2, (person[11][1]+person[12][1])/2  # hip 중점
       
       # height 구하기 (m과 n 사이의 거리)
       height =  (((n[0]-m[0])**2) + ((n[1]-m[1])**2)) ** 0.5
  
       if height > max_height:
         max_height = height
         adult_index = i

    i += 1

  return adult_index 


In [6]:
def sub(x1,x2):
  return x1[0] - x2[0],x1[1] - x2[1]

In [7]:
# keypoint별 벡터 크기(속력) 구하기
def keypoint_vector(frame, previous_keypoints, current_keypoints, confidence_threshold):
  
  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  c_shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  p_shaped = np.squeeze(np.multiply(previous_keypoints, [y,x,1]))
  
  # 이전 프레임과 현재 프레임의 Adult keypoints
  p_adult_keypoints = p_shaped[separate_adult(frame, previous_keypoints, 0.3)]
  c_adult_keypoints = c_shaped[separate_adult(frame, current_keypoints, 0.3)]

  # 벡터 차이 구하기
  vector = np.zeros((17,2)) 
  for i in range(17):
    # confidence 역치 이상일 때만
    if (c_adult_keypoints[i][2] >= confidence_threshold) and (p_adult_keypoints[i][2] >= confidence_threshold):
      vector[i][0] = c_adult_keypoints[i][0] - p_adult_keypoints[i][0]
      vector[i][1] = c_adult_keypoints[i][1] - p_adult_keypoints[i][1]
    else:
      vector[i] = [0,0]

  return vector
  

In [8]:
def relationKeypoint(frame,current_keypoints,confidence_threshold,distanceArray):
  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  
  # 이전 프레임과 현재 프레임의 Adult keypoints
  adult_keypoints = shaped[separate_adult(frame, current_keypoints, 0.3)]

  # 벡터 차이 구하기
  vector = np.zeros((22,1)) 
  #f4
  d4 = distanceArray[0] + distanceArray[2] + distanceArray[3] + distanceArray[5] +distanceArray[6]
  if d4 != 0:
    vector[0],vector[1] = sub(adult_keypoints[0],adult_keypoints[1])/d4
    vector[2],vector[3] = sub(adult_keypoints[6],adult_keypoints[1])/d4
    vector[4],vector[5] = sub(adult_keypoints[7],adult_keypoints[1])/d4
  else:
    vector[0],vector[1],vector[2],vector[3],vector[4],vector[5] = None,None,None,None,None,None
  #f5
  d5 = distanceArray[2] + distanceArray[3] + distanceArray[9] + distanceArray[11] 
  if d5 != 0:
    vector[6],vector[7] = sub(adult_keypoints[1],adult_keypoints[8])/d5
    vector[8],vector[9] = sub(adult_keypoints[7],adult_keypoints[8])/d5
    vector[10],vector[11] = sub(adult_keypoints[13],adult_keypoints[8])/d5
  else:
    vector[6],vector[7],vector[8],vector[9],vector[10],vector[11]= None,None,None,None,None,None
  #f6
  d6 = distanceArray[5] + distanceArray[6] + distanceArray[10] + distanceArray[12] 
  if d6 != 0:
    vector[12],vector[13] = sub(adult_keypoints[0],adult_keypoints[9])/d6
    vector[14],vector[15] = sub(adult_keypoints[6],adult_keypoints[9])/d6
    vector[16],vector[17] = sub(adult_keypoints[12],adult_keypoints[9])/d6
  else:
    vector[12],vector[13],vector[14],vector[15],vector[16],vector[17]= None,None,None,None,None,None
  #f7
  d7 = distanceArray[2] + distanceArray[3] + distanceArray[5] + distanceArray[6] 
  if d7 != 0:
    vector[18],vector[19] = sub(adult_keypoints[7],adult_keypoints[0])/d7
    vector[20],vector[21] = sub(adult_keypoints[6],adult_keypoints[0])/d7
  else:
    vector[18],vector[19],vector[20],vector[21]= None,None,None,None

  return vector


In [9]:
import math

def dist(x1,x2) -> int:
  x = x1[0]-x2[0]
  y = x1[1]-x2[1]
  return math.sqrt(x**2 + y**2)

In [10]:
def distanceVector(norm_vector):
  distanceArray = np.zeros((13,1))
  
  distanceArray[0] = dist((norm_vector[0],norm_vector[1]),(norm_vector[2],norm_vector[3]))
  distanceArray[1] = dist((norm_vector[2],norm_vector[3]),(norm_vector[6],norm_vector[7]))
  distanceArray[2] = dist((norm_vector[6],norm_vector[7]),(norm_vector[10],norm_vector[11]))
  distanceArray[3] = dist((norm_vector[10],norm_vector[11]),(norm_vector[14],norm_vector[15]))
  distanceArray[4] = dist((norm_vector[2],norm_vector[3]),(norm_vector[4],norm_vector[5]))
  distanceArray[5] = dist((norm_vector[4],norm_vector[5]),(norm_vector[8],norm_vector[9]))
  distanceArray[6] = dist((norm_vector[8],norm_vector[9]),(norm_vector[12],norm_vector[13]))
  distanceArray[7] = dist((norm_vector[2],norm_vector[3]),(norm_vector[18],norm_vector[19]))
  distanceArray[8] = dist((norm_vector[2],norm_vector[3]),(norm_vector[16],norm_vector[17]))
  distanceArray[9] = dist((norm_vector[18],norm_vector[19]),(norm_vector[22],norm_vector[23]))
  distanceArray[10] = dist((norm_vector[16],norm_vector[17]),(norm_vector[20],norm_vector[21]))
  distanceArray[11] = dist((norm_vector[22],norm_vector[23]),(norm_vector[26],norm_vector[27]))
  distanceArray[12] = dist((norm_vector[20],norm_vector[21]),(norm_vector[24],norm_vector[25]))

  return distanceArray

In [11]:
def normalize(frame, previous_keypoints, current_keypoints, confidence_threshold) :
  norm_vector = np.zeros((28,1))
  distanceArray = np.zeros((13,1))
  relationVector = np.zeros((22,1))
  final_vector = np.zeros((50,1))
  vector = keypoint_vector(frame, previous_keypoints, current_keypoints, 0.3)
  #head
  norm_vector[0],norm_vector[1] = (vector[1][0]+vector[2][0])//2,(vector[1][1]+vector[2][1])//2
  #neck
  norm_vector[2],norm_vector[3] = (norm_vector[0]+vector[5][0]+ vector[6][0])//3,(norm_vector[1]+vector[5][1]+ vector[6][1])//3

  #norm_vector로 옮기기 (14포인트)
  for i,points in enumerate(vector):
    x_idx,y_idx = (i-3)*2,((i-3)*2)+1
    norm_vector[x_idx] = points[0]
    norm_vector[y_idx] = points[1]
  
  # 신체 거리 구하기
  
  distanceArray = distanceVector(norm_vector)

  #변위 벡터 계산
  
  #f1
  d1 = distanceArray[0] + distanceArray[3] + distanceArray[6] + distanceArray[11] +distanceArray[12]
  if d1 != 0:
    norm_vector[0],norm_vector[1] = norm_vector[0]/d1,norm_vector[1]/d1
    norm_vector[12],norm_vector[13] = norm_vector[12]/d1,norm_vector[13]/d1
    norm_vector[14],norm_vector[15] = norm_vector[14]/d1,norm_vector[15]/d1
    norm_vector[24],norm_vector[25] = norm_vector[24]/d1,norm_vector[25]/d1
    norm_vector[26],norm_vector[27] = norm_vector[26]/d1,norm_vector[27]/d1
  else:
    norm_vector[0],norm_vector[1],norm_vector[12],norm_vector[13],norm_vector[14],norm_vector[15],norm_vector[24],norm_vector[25],norm_vector[26],norm_vector[27]=None,None,None,None,None,None,None,None,None,None

  #f2
  d2 = distanceArray[0] + distanceArray[2] + distanceArray[5] + distanceArray[9] +distanceArray[10]
  if d2 != 0:
    norm_vector[2],norm_vector[3] = norm_vector[2]/d2,norm_vector[3]/d2
    norm_vector[8],norm_vector[9] = norm_vector[8]/d2,norm_vector[9]/d2
    norm_vector[10],norm_vector[11] = norm_vector[10]/d2,norm_vector[11]/d2
    norm_vector[20],norm_vector[21] = norm_vector[20]/d2,norm_vector[21]/d2
    norm_vector[22],norm_vector[23] = norm_vector[22]/d2,norm_vector[23]/d2
  else:
    norm_vector[2],norm_vector[3],norm_vector[8],norm_vector[9],norm_vector[10],norm_vector[11],norm_vector[20],norm_vector[21],norm_vector[22],norm_vector[23]= None,None,None,None,None,None,None,None,None,None

  #f3
  d3 = distanceArray[1] + distanceArray[4] + distanceArray[9] + distanceArray[10] 
  if d3 != 0:
    norm_vector[4],norm_vector[5] = norm_vector[4]/d3,norm_vector[5]/d3
    norm_vector[6],norm_vector[7] = norm_vector[6]/d3,norm_vector[7]/d3
    norm_vector[16],norm_vector[17] = norm_vector[16]/d3,norm_vector[17]/d3
    norm_vector[18],norm_vector[19] = norm_vector[18]/d3,norm_vector[19]/d3
  else:
    norm_vector[4],norm_vector[5],norm_vector[6],norm_vector[7],norm_vector[16],norm_vector[17],norm_vector[18],norm_vector[19] = None,None,None,None,None,None,None,None
  
  relationVector = relationKeypoint(frame,current_keypoints,confidence_threshold,distanceArray)

  final_vector = np.concatenate((norm_vector, relationVector), axis=0)
  return final_vector

In [12]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from google.colab.patches import cv2_imshow
import os


path = '/content/gdrive/MyDrive/1010_동영상_쓰다듬기'
file_list = os.listdir(path)
n=1
for i in range(len(file_list)):
  address = '/content/gdrive/MyDrive/1010_동영상_쓰다듬기/' + file_list[i]
  cap = cv2.VideoCapture(address)
  prevImg = None
  current_keypoints = None
  previous_keypoints = None
  result = None
  input = np.zeros((51,1))

  while cap.isOpened():

      ret, frame = cap.read()
      # Resize image
      if frame is None:
        break
      else: 
        img = frame.copy()
      img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
      input_img = tf.cast(img, dtype=tf.int32)
      
      if prevImg is None:    
        prevImg = input_img

        # Detection section
        results1 = movenet(prevImg)

        # 17개의 키포인트: nose, Leye, Reye, Lear, Rear, Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip, Lknee, Rknee, Lankle, Rankle
        previous_keypoints = results1['output_0'].numpy()[:,:,:51].reshape((6,17,3))

      else:
        currentImg = input_img

        # Detection section
        results2 = movenet(currentImg)

        # 17개의 키포인트: nose, Leye, Reye, Lear, Rear, Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip, Lknee, Rknee, Lankle, Rankle
        current_keypoints = results2['output_0'].numpy()[:,:,:51].reshape((6,17,3))

        # 다음 프레임을 위한 프레임 이월
        prevImg = currentImg

        #프레임마다 input계산
        vector = normalize(frame, previous_keypoints, current_keypoints, 0.2)
        #flag=[[0.]] #때리는 값
        flag=[[1.]] #쓰다듬는 값
        vectors = np.concatenate((vector, flag))

        #프레임 별로 합치기
        input = np.append(input, vectors , axis = 1)

      if cv2.waitKey(10) & 0xFF==ord('q'):
          break

  result = np.delete(input,0,axis=1)
  #print(result)
  #print('*'*50)
  df = pd.DataFrame(result)
  name='input_쓰다듬기'+str(n)
  df.to_csv('/content/gdrive/MyDrive/csv_쓰다듬기/'+name+'.csv', index=False)

  n+=1

  cap.release()
  cv2.destroyAllWindows()
